In [ ]:
import os, os.path
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0' 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device='/gpu:0'
import random
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.keras.backend.floatx()
from collections import deque
import itertools
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from libs.utils import *
from libs.generate_boxes import *
from libs.dqn import *

num_episode = 1500
global_step = 0
tr_l, h_fill, tr_r, avg_loss_l,history_eps,used_boxes_eps  = [],[],[],[],[],[]
N_MDD = 7
K = 4# 선택할 박스 수
n_candidates =4 # K보다 커야 함

# generating boxes
# boxes_multi1,gt_pos1 = generation_3dbox(case_size=[[20,20,20]], min_s = 1, N_mdd=N_MDD)#, is_prediv='h')

boxes_multi1 = [np.array([[20, 20,  4],
         [20,  4,  4],
         [20,  4,  4],
         [20,  4,  4],
         [20,  4,  4],
         [20,  4,  4],
         [20, 20,  4],
         [20, 20,  4],
         [20, 20,  4]])]
gt_pos1 = [np.array([[ 0,  0,  0],
         [ 0,  0,  4],
         [ 0,  4,  4],
         [ 0,  8,  4],
         [ 0, 12,  4],
         [ 0, 16,  4],
         [ 0,  0,  8],
         [ 0,  0, 12],
         [ 0,  0, 16]])]

boxes_multi2 = [np.array([[20, 20,  5],
        [ 4, 20,  5],
        [ 4, 20,  5],
        [ 4, 20,  5],
        [ 4, 20,  5],
        [ 4, 20,  5],
        [10, 20,  5],
        [10, 20,  5],
        [20, 20,  5]])]

gt_pos2 = [np.array([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 4,  0,  5],
        [ 8,  0,  5],
        [12,  0,  5],
        [16,  0,  5],
        [ 0,  0, 10],
        [10,  0, 10],
        [ 0,  0, 15]])]
# idx = np.lexsort((gt_pos2[:,1],gt_pos2[:,0],gt_pos2[:,2]))
# gt_pos2 = gt_pos2[idx]
# boxes_multi2 = np.array(boxes_multi2)[idx]

num_max_boxes = max(len(boxes_multi1[0]), len(boxes_multi2[0]))#len(boxes_multi[0]) #N_MDD+3# 
num_max_remain = num_max_boxes# - K
print('num_max_boxes',num_max_boxes,'num_max_remain',num_max_remain)

env=Bpp3DEnv()
agent = DQNAgent( L=20, B=20, H=20, n_remains = num_max_remain, n_loading = K, 
                 lr=1e-4, exp_steps=900, train_st = 500, memory_len=1000, update_target_rate = 50,
                net = 'DDQN')

In [ ]:
for e in range(1,num_episode):
    st=time.time()
    
    if num_episode>1000 and agent.epsilon <= 0.01:
        agent.epsilon = 0
    
    if e%2 == 1: boxes_multi,gt_pos = boxes_multi1.copy(), gt_pos1.copy()
    else: boxes_multi,gt_pos = boxes_multi2.copy(), gt_pos2.copy()
    
    env.reset()# env 초기화
    done = False
    step = 0
    history, h_load, h_remain_size, h_load_size = [],[],[],[]
    t_history, t_load, t_remain_size, t_load_size = [],[],[],[]
    used_boxes, pred_pos = [],[]
    
    #boxes_all = np.array(boxes_multi)[e%100].copy()
    boxes_all = np.array(boxes_multi)[0].copy()
    r_boxes = boxes_all.copy() #single case
    
    if agent.epsilon > agent.epsilon_end and len(agent.memory)>=agent.train_start:
        agent.epsilon -= agent.epsilon_decay_step

    while not done:#중박스에 대해 반복##############
        state = env.container_s.copy()
        state_h = env.container_h.copy()
        global_step += 1
        step += 1
        k = min(K, len(r_boxes))
        selected = cbn_select_boxes(r_boxes[:n_candidates], k) #r_boxes의 앞에서부터 n_candidates 만큼의 상자 중 k개의 중박스 선택 
        #selected = r_boxes.reshape((-1,1,3))
        s_order = get_selected_order(selected, k)
        
        num_loaded_c, loading_size_c, loading_pos_c, next_state_c, next_h_c, loaded_mh_c = get_selected_location(s_order, state, state_h, env.height)
        in_state, in_r_boxes, in_loading = raw2input(state_h, len(num_loaded_c), r_boxes, num_max_remain, K, loading_size_c, env.height) # input 변환 
        
        # action 선택 수가 1일 때 action 스킵
        if len(num_loaded_c) == 1:
            action_idx = 0
        else: # 선택 가능
            # action 실행
            action_idx = agent.get_action(in_state, np.array(loaded_mh_c), in_r_boxes, in_loading)
        
        # 다음 step ########################
        # env의 다음 state
        env.step(next_state_c[action_idx], next_h_c[action_idx]) 
        # r_boxes 업데이트
        num_loaded_box = num_loaded_c[action_idx]
        if num_loaded_box != 0: # 적입된 박스가 하나 이상 있는 경우 -> append
            new_used_boxes = loading_size_c[action_idx] # 기존 적재 박스, 기존 + 새 박스 -> 새로 적재된 박스 
            r_boxes = get_remain(new_used_boxes, r_boxes) # 남은 박스 업데이트 = 남은 박스 - 새로 적재 박스
            # 사용된 박스, 위치 업데이트
            used_boxes = used_boxes + loading_size_c[action_idx] #사용된 박스 업데이트
            pred_pos = pred_pos + loading_pos_c[action_idx] #예측된 위치 업데이트    
        
        else: # 적입된 박스가 없을 경우 - action = zero, state=t_state, 해당 박스들 모두 스킵
            r_boxes = get_remain(s_order[action_idx], r_boxes) # 남은 박스 업데이트 = 남은 박스 - 선택된 박스
        if len(r_boxes) == 0 or np.sum(env.container_h != env.height) == 0: # 더 이상 적입할 적입할 수 없는 경우(박스 다 사용 or 다 채워짐)
            done = True
        
        # action 선택 수가 1이 이상이고 마지막 박스가 적입되었을 때 append 
        if len(num_loaded_c) != 1 or done:
            # (s, a) append ########################
            history.append(in_state[action_idx]) # 적재 전 상태
            h_load.append(loaded_mh_c[action_idx]) # 적재된 상태
            h_remain_size.append(in_r_boxes[action_idx]) # 선택 후 전처리된 남은 박스 사이즈
            h_load_size.append(in_loading[action_idx]) # 적재된 박스 사이즈

            # (s') append ########################
            t_state = next_state_c[action_idx].copy() # 다음 state 
            t_state_h = next_h_c[action_idx].copy() 
            if done: #마지막 박스까지 채워졌을 경우 or 더 이상 적입할 수 없을 경우
                in_t_history = t_state_h.reshape((1, env.length, env.breadth, 1))
                loaded_mh_c = np.zeros((1, env.length, env.breadth, 2)) #(N, L, B, 2)
                in_t_remains = np.zeros((1, env.length, env.breadth, num_max_remain))
                in_t_loading = np.zeros((1, env.length, env.breadth, K))
            else:
                k = min(K, len(r_boxes))
                selected = cbn_select_boxes(r_boxes[:n_candidates], k) # 업데이트된 남은 박스들 중 중박스 선택
                #selected = r_boxes.reshape((-1,1,3))
                s_order = get_selected_order(selected, k) # 정렬 선택
                num_loaded_c, loading_size_c, loading_pos_c, next_state_c, next_h_c, loaded_mh_c = get_selected_location(s_order, t_state, t_state_h, env.height)
                in_t_history, in_t_remains, in_t_loading = \
                    raw2input(t_state_h, len(num_loaded_c), r_boxes, num_max_remain,  K, loading_size_c, env.height)
                
            t_history.append(in_t_history) # 적재 전 상태
            t_load.append(np.array(loaded_mh_c)) # 적재된 상태
            t_remain_size.append(in_t_remains) # 선택 후 전처리된 남은 박스 사이즈
            t_load_size.append(in_t_loading) # 적재된 박스 사이즈
                
        
        # terminal reward 계산 -> replay memory에 추가 
        if done:
            # terminal reward 
            avg_tr = 0 if len(tr_r)==0 else np.mean(tr_r)
            terminal_reward = (env.terminal_reward())**3 # - avg_tr
            tr_l.append(terminal_reward)
            h_fill.append(env.terminal_reward())
            if agent.epsilon >= 0.7: tr_r.append(env.terminal_reward())
            
            a_repeate = 1#6 if env.terminal_reward() ==1.0 else 1
            # append samples
            is_last = False 
            N = len(history)
            for i in range(N):
                if i == N-1: is_last=True
                reward=(0.99**(N-i-1))*terminal_reward #step reward
                for a in range(a_repeate):
                    agent.append_sample(history[i], h_load[i], h_remain_size[i], h_load_size[i], reward, is_last,
                                        t_history[i], t_load[i], t_remain_size[i], t_load_size[i])
            if global_step > agent.train_start:
                agent.draw_tensorboard(terminal_reward, env.terminal_reward(), step, e)
                
        # 학습 시작
        if len(agent.memory) >= agent.train_start:
            agent.train_model()
            if global_step % agent.update_target_rate == 0: # target model 업데이트
                print('update',e)
                agent.update_target_model()
            avg_loss_l.append(agent.avg_loss/float(step))
    
        #vis_box(used_boxes,pred_pos)
        #print(env.container_h)
    #if global_step > agent.train_start:
    #    agent.draw_tensorboard(score, step, e)
    log = "=====episode: {:5d} | ".format(e)
    log += "memory length: {:5d} | ".format(len(agent.memory))
    log += "epsilon: {:.3f} | ".format(agent.epsilon)
    log += "reward(): {:.3f}, {:.3f}| ".format(env.terminal_reward(), terminal_reward)
    #log += "t_reward: {:.3f} | ".format(t_reward)
    #log += "q avg : {:3.2f} | ".format(agent.avg_q_max / float(step))
    log += "avg loss : {:6f} ".format(agent.avg_loss / float(step))
    log += "time: {:.3f}".format(time.time()-st)
    print(log)
    agent.avg_q_max, agent.avg_loss = 0, 0
    
    #used_boxes_eps.append(used_boxes)
    #history_eps.append(history)
    # 1000 에피소드마다 모델 저장
    if e % 1000 == 0:
        agent.model.save_weights("save_model/model", save_format="tf")
    
#print('total training time', time.time()-st)

In [ ]:
%matplotlib inline

plt.figure(figsize=(20,3))
plt.plot(h_fill,'g-')
plt.ylim(-0.2,1.3)
plt.grid()

plt.figure(figsize=(20,3))
plt.plot(avg_loss_l,'b-')
plt.grid()
plt.show()

In [ ]:
%matplotlib notebook
# %matplotlib inline
vis_box(used_boxes,pred_pos)

In [ ]:
%matplotlib notebook
# %matplotlib inline
vis_box(np.array(boxes_multi)[0],np.array(gt_pos)[0])